# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db UK_food --collection establishments --file establishments.json --jsonArray`

In [72]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [73]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [74]:
# confirm that our new database was created
databases = mongo.list_database_names()
print(databases)

['UK_food', 'admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardebDB', 'gardenDB2', 'local', 'met', 'petsitly_marketing', 'test', 'travel_db']


In [75]:
# assign the uk_food database to a variable name
food_db = mongo['UK_food']

In [76]:
# review the collections in our new database

print("Collections in UK_food:", food_db.list_collection_names())

Collections in UK_food: ['establishments']


In [77]:
# review the collections in our new database
#This seems repetitive. see above cell. It already reviewed the collection.

In [78]:
# review a document in the establishments collection
restaurants = food_db['establishments']

for doc in restaurants.find().limit(1):
    pprint(doc)


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('680c3e0dc829a101f29e9b4c'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [79]:
# assign the collection to a variable
restaurants = food_db['establishments']
# review the first 5 documents in the collection
for doc in restaurants.find().limit(5):
    pprint(doc)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('680c3e0dc829a101f29e9b4c'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [80]:
# Create a dictionary for the new restaurant data
penang_flavours = {}


In [81]:
# Insert the new restaurant into the collection``
insert_penang_flavours = restaurants.insert_one(penang_flavours)

In [82]:
# Check that the new restaurant was inserted

latest_restaurant = restaurants.find_one({"_id": insert_penang_flavours.inserted_id})
pprint(latest_restaurant)



{'_id': ObjectId('680c7b1df443b0272529b7e0')}


In [83]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

bizz_types = restaurants.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"_id": 0,"BusinessTypeID": 1, "BusinessType": 1})
pprint(bizz_types)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [96]:
# Update the new restaurant with the correct BusinessTypeID
restaurants.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}},
    upsert=True
)

UpdateResult({'n': 1, 'upserted': ObjectId('680c7c00708ea0aaa355d3eb'), 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [109]:
# Confirm that the new restaurant was updated
check_update = restaurants.find_one({"BusinessName": "Penang Flavours"})
pprint(check_update)


{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('680c7c00708ea0aaa355d3eb')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [98]:
# Find how many documents have LocalAuthorityName as "Dover"
check_dover = restaurants.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {check_dover}")


Number of documents with LocalAuthorityName 'Dover': 994


In [99]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dover = restaurants.delete_many({"LocalAuthorityName": "Dover"})
# Confirm that the documents were deleted
deleted_count = delete_dover.deleted_count
print(f"Number of documents deleted with LocalAuthorityName 'Dover': {deleted_count}")

Number of documents deleted with LocalAuthorityName 'Dover': 994


In [100]:
# Check if any remaining documents include Dover
remaining_dover = restaurants.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {remaining_dover}")


Number of remaining documents with LocalAuthorityName 'Dover': 0


In [ ]:
# Check that other documents remain with 'find_one'
#using Penang Flavours as an example
remaining = restaurants.find_one({"BusinessName": "Penang Flavours"})
pprint(remaining)

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('680c7c00708ea0aaa355d3eb')}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [113]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128

for doc in restaurants.find({}, {"geocode.longitude": 1, "geocode.latitude": 1}):
    geocode = doc.get("geocode", {})
    if isinstance(geocode.get("longitude"), str):
        restaurants.update_one(
            {"_id": doc["_id"]},
            {"$set": {
                "geocode.longitude": Decimal128(geocode["longitude"]),
                "geocode.latitude": Decimal128(geocode["latitude"])
            }}
        )
print("Geocode longitude and latitude updated to Decimal128.")

Geocode longitude and latitude updated to Decimal128.


Use `update_many` to convert `RatingValue` to integer numbers.

In [114]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
restaurants.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [116]:
# Change the data type from String to Integer for RatingValue
ratings_string_int = restaurants.find({"RatingValue": {"$type": "string"}})
for doc in ratings_string_int:
    if doc["RatingValue"] is not None:
        restaurants.update_one(
            {"_id": doc["_id"]},
            {"$set": {"RatingValue": int(doc["RatingValue"])}}
        )
print("RatingValue updated to Integer.")    

RatingValue updated to Integer.


In [125]:
# Check that the coordinates and rating value are now numbers

for doc in restaurants.find():
    rating = doc.get("RatingValue")
    if isinstance(rating, int):
        print(f"RatingValue is an integer: {rating}")
    elif isinstance(rating, Decimal128):
        print(f"RatingValue is a Decimal128: {rating}")
    else:
        print("RatingValue is not a integer or number.")


RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 4
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 4
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 4
RatingValue is an integer: 4
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 2
RatingValue is an integer: 3
RatingValue is an integer: 4
RatingValue is an integer: 5
RatingValue is an integer: 4
RatingValue is an integer: 5
RatingValue is an integer: 3
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is an integer: 5
RatingValue is